# Going Deeper 07 Machine translation by LSTM
###### 온라인 2기 코어 박수경

## 라이브러리 및 데이터 로딩  

In [1]:
import tensorflow
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

import re
import string

import numpy as np
import pandas as pd
import os

print(tensorflow.__version__)

2.6.0


In [2]:
file_path = os.getenv('HOME')+'/aiffel/translator_seq2seq/data/fra.txt'
lines = pd.read_csv(file_path, names=['eng', 'fra', 'cc'], sep='\t')

lines = lines.sample(frac=1).reset_index(drop=True) # 섞고 인덱스 리셋하기 -섞지않은 데이터는 문자열순서대로 정렬되어 있으므로.

lines = lines[['eng', 'fra']][:33000] # 사용하는 샘플 개수
lines.sample(5) #샘플 5개 출력

,eng,fra
30749,Don't believe everything people tell you.,Ne croyez pas à tout ce que les gens vous disent.
20014,He just texted me.,Il vient de me faire parvenir un texto.
23864,Did you go out?,Es-tu sortie ?
22111,I'm the one who killed Tom.,Je suis celui qui a tué Tom.
18342,Don't turn off your computer.,N'éteins pas ton ordinateur.


seq2seq 번역모델은 인코더, 디코더 두 부분으로 나뉘어져 있다.  
![](https://d3s0tskafalll9.cloudfront.net/media/images/E-15-4.seq2seq.max-800x600.jpg)
인코더에서 사용하는 시퀀스와는 달리 디코더는 시퀀스의 처음과 끝에 각각을 알리는 토큰이 필요하다. 
기본적인 정제를 다 거치고 30000개는 학습 셋, 3000개는 테스트 셋으로 사용한다. 

## 정제, 정규화 및 기타 전처리  

글자 단위가 아닌 단어 단위의 번역기를 하기 위해서 전처리를 한다.  
Punctuation을단어와 분리하고, 대문자를 모두 소문자로 바꾼다. 그 후 띄어쓰기 단위로 문장을 토큰화한다. 

필요한 전처리 과정을 함수화한다.

In [3]:
import string

string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [4]:
punctuation = '!"#$%&\'()*+,-./:;=?@[\\]^_`{|}~'

In [5]:
start_token = '<s>'
end_token = '<e>'

In [6]:
# 1. 띄어쓰기를 기준으로 단어들을 분리
# 2. 구두점을 단어와 분리 (문장의 끝을 알리는 문장부호)
# 3. 모두 소문자화하기 * 궁금한 점. 고유명사는 어떻게???

def polish_up(lines, token = False ):
        
    for i in range(len(lines)):
        lines[i] = str(lines[i]).lower()# 소문자
        lines[i] = ''.join(char for char in lines[i] if char not in punctuation) # 구두점(Punctuation)을 단어와 분리하기
          
        
    if token == True:
        for i in range(len(lines)):
            lines[i] = start_token +' '+ lines[i] +' '+ end_token     
    
    return lines


In [7]:
polish_up(lines.eng)
polish_up(lines.fra, token = True )

0        <s> je ne savais pas quelles pouvaient faire ç...
1          <s> voulezvous que jespionne tom pour vous  <e>
2        <s> je dois retoucher mes vêtements parce que ...
3        <s> «entre seulement et dis au patron que tu v...
4                <s> il sarrêta pour regarder laffiche <e>
                               ...                        
32995    <s> les enfants ont parfois peur dans le noir <e>
32996    <s> puisje aller vous chercher quoi que ce soi...
32997                 <s> la vieille femme est médecin <e>
32998                  <s> nous avons bu toute la nuit <e>
32999          <s> vous avez atteint votre destination <e>
Name: fra, Length: 33000, dtype: object

In [8]:
lines.eng.sample(10)
lines.fra.sample(10)

7022     <s> je ne pense pas quaucun de nous deux ne ve...
20006    <s> javais hâte de voir une vue panoramique du...
25457                 <s> je nai rien dautre à ajouter <e>
22709    <s> je suis habituée à ce que tom me crie tout...
29508    <s> je te promets que jexpliquerai tout plus t...
10251         <s> pourquoi ne discutestu pas avec lui  <e>
17376                <s> il est difficile dêtre parent <e>
6784                             <s> lâchezmoi le bras <e>
25582             <s> je sais que ça doit être un choc <e>
2868                   <s> je sais que vous lavez fait <e>
Name: fra, dtype: object

## Tokenize  


### 텍스트를 토큰으로 인코딩  

참고 : https://wikidocs.net/31766

In [9]:
eng_tokenizer = Tokenizer(char_level=False)   # 문자 단위로 Tokenizer를 생성합니다. 
eng_tokenizer.fit_on_texts(lines.eng)               # 33000개의 행을 가진 eng의 각 행에 토큰화를 수행
input_text = eng_tokenizer.texts_to_sequences(lines.eng)    # 단어를 숫자값 인덱스로 변환하여 저장
input_text[:3]

[[1, 55, 30, 42, 86, 9, 8],
 [9, 2, 27, 14, 3, 2525, 29, 7, 20, 2],
 [1, 16, 3, 2929, 19, 467, 298, 76, 285, 849]]

In [10]:
encoder_input = input_text


In [11]:
fra_tokenizer = Tokenizer(char_level=False)   # 문자 단위로 Tokenizer를 생성합니다. 
fra_tokenizer.fit_on_texts(lines.fra)                 # 50000개의 행을 가진 fra의 각 행에 토큰화를 수행
target_text = fra_tokenizer.texts_to_sequences(lines.fra)     # 단어를 숫자값 인덱스로 변환하여 저장
target_text[:3]

[[1, 3, 9, 289, 5, 413, 2815, 27, 29, 2],
 [1, 251, 6, 7632, 12, 25, 11, 2],
 [1, 3, 124, 7633, 95, 529, 556, 6, 23, 285, 37, 1018, 2]]

In [12]:
#토큰 제거

decoder_input = [[char for char in line if char != 1] for line in target_text]
decoder_target =[[char for char in line if char != 2] for line in target_text]

In [13]:
eng_vocab_size = len(eng_tokenizer.word_index) + 1
fra_vocab_size = len(fra_tokenizer.word_index) + 1
print('영어 단어장의 크기 :', eng_vocab_size)
print('프랑스어 단어장의 크기 :', fra_vocab_size)

영어 단어장의 크기 : 8346
프랑스어 단어장의 크기 : 15969


In [14]:
n_of_val = 3000

encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]


## Embedding layer (각 단어를 임베딩 층을 사용하여 벡터화)  

In [15]:
emb_dim = 256
hidden_size = 512

In [16]:
from tensorflow.keras.layers import Input, Embedding, Masking

# 인코더에서 사용할 임베딩 층 사용 예시
encoder_inputs = Input(shape=(None,), name='incoder_input')
enc_emb =  Embedding(eng_vocab_size, emb_dim)(encoder_inputs)
encoder_lstm = LSTM(hidden_size , return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
encoder_states = [state_h, state_c]

In [19]:
decoder_inputs = Input(shape=(None, ), name='decoder_input')
dec_emb =  Embedding(fra_vocab_size, emb_dim)(decoder_inputs)

decoder_lstm = LSTM(hidden_size, return_sequences = True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state = encoder_states)


decoder_softmax_layer = Dense(fra_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

In [20]:
max_eng_seq_len = 11
max_fra_seq_len = 20



## 모델링  

label이 integer 값이므로 categorical entropy loss가 아닌 sparse categorical entropy loss를 사용한다.


In [21]:
encoder_model = Model(inputs = encoder_inputs, outputs = encoder_states)
encoder_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
incoder_input (InputLayer)   [(None, None)]            0         
_________________________________________________________________
embedding (Embedding)        (None, None, 256)         2136576   
_________________________________________________________________
lstm (LSTM)                  [(None, 512), (None, 512) 1574912   
Total params: 3,711,488
Trainable params: 3,711,488
Non-trainable params: 0
_________________________________________________________________


In [22]:
# 이전 time step의 hidden state를 저장하는 텐서
decoder_state_input_h = Input(shape=(512,))
# 이전 time step의 cell state를 저장하는 텐서
decoder_state_input_c = Input(shape=(512,))
# 이전 time step의 hidden state와 cell state를 하나의 변수에 저장
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# decoder_states_inputs를 현재 time step의 초기 상태로 사용.
# 구체적인 동작 자체는 def decode_sequence()에 구현.
dec_emb_2 = Embedding(fra_vocab_size, emb_dim)(decoder_inputs)

decoder_outputs, state_h, state_c = decoder_lstm(dec_emb_2, initial_state = decoder_states_inputs)
# 현재 time step의 hidden state와 cell state를 하나의 변수에 저장.
decoder_states = [state_h, state_c]

In [23]:
#디코더의 출력층 재설계

decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs] + decoder_states)
decoder_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
decoder_input (InputLayer)      [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, None, 256)    4088064     decoder_input[0][0]              
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 512)]        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 512)]        0                                            
____________________________________________________________________________________________

## Model evaluation  

In [24]:
fra_tokenizer.word_index['<s>'] = 1
fra_tokenizer.word_index['<e>'] = 2


In [25]:

eng_to_idx = eng_tokenizer.word_index
fra_to_idx = fra_tokenizer.word_index
idx_to_ng = eng_tokenizer.index_word
idx_to_fra = fra_tokenizer.index_word

In [26]:
fra_tokenizer.index_word[1] = '<s>'
fra_tokenizer.index_word[2] = '<e>'

In [27]:
fra_tokenizer.index_word

{1: '<s>',
 2: '<e>',
 3: 'je',
 4: 'de',
 5: 'pas',
 6: 'que',
 7: 'à',
 8: 'la',
 9: 'ne',
 10: 'le',
 11: 'vous',
 12: 'tom',
 13: 'il',
 14: 'est',
 15: 'un',
 16: 'ce',
 17: 'a',
 18: 'tu',
 19: 'nous',
 20: 'en',
 21: 'les',
 22: 'une',
 23: 'jai',
 24: 'suis',
 25: 'pour',
 26: 'me',
 27: 'faire',
 28: 'cest',
 29: 'ça',
 30: 'des',
 31: 'plus',
 32: 'elle',
 33: 'dans',
 34: 'tout',
 35: 'qui',
 36: 'te',
 37: 'du',
 38: 'ma',
 39: 'se',
 40: 'au',
 41: 'fait',
 42: 'avec',
 43: 'si',
 44: 'mon',
 45: 'veux',
 46: 'et',
 47: 'sont',
 48: 'quil',
 49: 'y',
 50: 'cette',
 51: 'très',
 52: 'votre',
 53: 'pense',
 54: 'être',
 55: 'cela',
 56: 'son',
 57: 'nest',
 58: 'sur',
 59: 'été',
 60: 'pourquoi',
 61: 'temps',
 62: 'dit',
 63: 'moi',
 64: 'était',
 65: 'peux',
 66: 'ils',
 67: 'lui',
 68: 'ici',
 69: 'chose',
 70: 'nai',
 71: 'sais',
 72: 'jamais',
 73: 'où',
 74: 'estce',
 75: 'comment',
 76: 'quelque',
 77: 'ton',
 78: 'toi',
 79: 'besoin',
 80: 'bien',
 81: 'vraiment',
 8

In [28]:
# 문장을 디코딩

def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)

    # 에 해당하는 원-핫 벡터 생성
    target_seq = np.zeros((1,1)) 
    target_seq[0, 0] = fra_to_idx['<s>']
    
    stop_condition = False
    decoded_sentence = ""

    # stop_condition이 True가 될 때까지 루프 반복
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # 예측 결과를 문자로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = idx_to_fra[sampled_token_index]

        # 현재 시점의 예측 문자를 예측 문장에 추가
        decoded_sentence += ' '+sampled_char

        # 에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_char == '<e>' or
           len(decoded_sentence) > 20):
            stop_condition = True

        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장     
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sentence

번역한 문장을 테스트한다.

In [38]:
import numpy as np
for seq_index in [3,50,100,300,1001]: # 입력 문장의 인덱스 (자유롭게 선택해 보세요)
    input_seq = encoder_input[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print(35 * "-")
    print('입력 문장:', lines.eng[seq_index])
    print('정답 문장:', lines.fra[seq_index][3:len(lines.fra[seq_index])-3]) 
    print('번역기가 번역한 문장:', decoded_sentence[:len(decoded_sentence)-3])

-----------------------------------
입력 문장: just go in and tell the boss you want a raise thats easier said than done
정답 문장:  «entre seulement et dis au patron que tu veux une augmentation» «plus facile à dire quà faire» 
번역기가 번역한 문장:  nécoutes nécoutes sérieusem
-----------------------------------
입력 문장: i think tom may be dead
정답 문장:  je crois que tom est peutêtre mort 
번역기가 번역한 문장:  herbert oiseau fo
-----------------------------------
입력 문장: can i see you in my office
정답 문장:  puisje vous voir dans mon bureau  
번역기가 번역한 문장:  limage montagne corall
-----------------------------------
입력 문장: your garden needs some attention
정답 문장:  votre jardin a besoin dun peu dattention 
번역기가 번역한 문장:  lextirpa jadmire criti
-----------------------------------
입력 문장: if you have something to say just say it
정답 문장:  si tu as quelque chose à dire disle 
번역기가 번역한 문장:  folie fascinée secréta


## Discussion

 번역의 질이 너무 별로였다.  
 추후 코드를 꼭 보완하고 싶다.
 
 - 토큰, 정제의 순서에 따라 계속 단어 딕셔너리가 제대로 만들어지지 않는 점.
 
 - 섞는 방법의 문제. 엉망으로 섞여버린 것 같다. 단어사전이 서로 인덱스가 매칭이 되지 않아 영 이상한 문장이 출력된 것 같다.
 
 - seq2seq 모델의 성능을 높이기 위한 하이퍼파라미터 조정. 시도 자체가 부족했다.  
 

## References

https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html (seq2seq 모델링)  
https://wikidocs.net/33793 (임베딩레이어 사용하기)  
https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer (케라스 제공의 텍스트 토크나이저)  
https://www.techiedelight.com/ko/remove-punctuations-string-python/ (구두점 제거)  

